In [2]:
import pandas as pd
import numpy as np
import sqlite3
import altair as alt
import matplotlib.pyplot as plt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
con = sqlite3.connect("switrs.sqlite")

query = """
    SELECT * FROM collisions WHERE county_location = 'los angeles' OR county_location = 'san diego'
    """

df = pd.read_sql_query(query, con, parse_dates = ["collision_date"])

dfc = df[["case_id", "county_location", "alcohol_involved", "collision_severity", "injured_victims", "collision_date", "collision_time"]]

In [4]:
dfi_la, dfi_sd = pd.DataFrame(), pd.DataFrame()

dfi_la["count"] = dfc.query("county_location == 'los angeles'")["injured_victims"].value_counts()
dfi_la["logcount"] = dfi_la["count"].apply(lambda x: np.log(x))
dfi_la["injured_victims"] = dfc.query("county_location == 'los angeles'")["injured_victims"].value_counts().index
dfi_la["county"] = "Los Angeles"

dfi_sd["count"] = dfc.query("county_location == 'san diego'")["injured_victims"].value_counts()
dfi_sd["logcount"] = dfi_sd["count"].apply(lambda x: np.log(x))
dfi_sd["injured_victims"] = dfc.query("county_location == 'san diego'")["injured_victims"].value_counts().index
dfi_sd["county"] = "San Diego"

dfi = pd.concat([dfi_la, dfi_sd])

injuries = alt.Chart(dfi).mark_bar(opacity = 0.75, width = 11).encode(
    x = alt.X("injured_victims:Q"),
    y = alt.Y("count:Q").stack(None).scale(type = "log"),
    color = "county"
).properties(width = 800, height = 300)

(injuries).display()

alt.Chart(...)

In [5]:
# same as above but scaled to total accidents per county

dfi_la, dfi_sd = pd.DataFrame(), pd.DataFrame()

dfi_la["count"] = dfc.query("county_location == 'los angeles'")["injured_victims"].value_counts()
dfi_la["pcount"] = dfi_la["count"] / len(dfc.query("county_location == 'los angeles'"))
dfi_la["logpcount"] = dfi_la["pcount"].apply(lambda x: np.log(x))
dfi_la["injured_victims"] = dfc.query("county_location == 'los angeles'")["injured_victims"].value_counts().index
dfi_la["county"] = "Los Angeles"

dfi_sd["count"] = dfc.query("county_location == 'san diego'")["injured_victims"].value_counts()
dfi_sd["pcount"] = dfi_sd["count"] / len(dfc.query("county_location == 'san diego'"))
dfi_sd["logpcount"] = dfi_sd["pcount"].apply(lambda x: np.log(x))
dfi_sd["injured_victims"] = dfc.query("county_location == 'san diego'")["injured_victims"].value_counts().index
dfi_sd["county"] = "San Diego"

dfi = pd.concat([dfi_la, dfi_sd])

injuries = alt.Chart(dfi).mark_bar(opacity = 0.75, width = 11).encode(
    x = alt.X("injured_victims:Q"),
    y = alt.Y("pcount:Q").stack(None).scale(type = "log"),
    color = "county"
).properties(width = 800, height = 300)

(injuries).display()

alt.Chart(...)

In [6]:
# average injuries per collision for each county

avgila = dfc.query("county_location == 'los angeles'")["injured_victims"].sum() / len(dfc.query("county_location == 'los angeles'"))
avgisd = dfc.query("county_location == 'san diego'")["injured_victims"].sum() / len(dfc.query("county_location == 'san diego'"))

print(avgila, avgisd)

0.5800240188644512 0.7679370271622641


In [7]:
# proportion of accidents with 0 injuries

nipla = len(dfc.query("county_location == 'los angeles' & injured_victims == 0")) / len(dfc.query("county_location == 'los angeles'"))
nipsd = len(dfc.query("county_location == 'san diego' & injured_victims == 0")) / len(dfc.query("county_location == 'san diego'"))

print(nipla, nipsd)

0.6025424932282581 0.45739325909827555


In [36]:
dfy = dfc.copy()

dfy["year"] = dfy["collision_date"].dt.year

dfy_la, dfy_sd = pd.DataFrame(), pd.DataFrame()

dfy_la["count"] = dfy.query("county_location == 'los angeles'").groupby("year")["case_id"].count() / len(dfy.query("county_location == 'los angeles'"))
dfy_la["county"] = "Los Angeles"

dfy_sd["count"] = dfy.query("county_location == 'san diego'").groupby("year")["case_id"].count() / len(dfy.query("county_location == 'san diego'"))
dfy_sd["county"] = "San Diego"

dfyp = pd.concat([dfy_la, dfy_sd]).reset_index().query("year != 2021")

dfyp_plot = alt.Chart(dfyp).mark_bar(opacity = 0.75, width = 36).encode(
    x = alt.X("year:Q"),
    y = alt.Y("count:Q").stack(None),
).properties(width = 800, height = 300)

dfyp_plot.display()

alt.Chart(...)

In [65]:
dfa = dfc.copy()
dfa["year"] = dfa["collision_date"].dt.year
dfa["alcohol_involved"] = dfa["alcohol_involved"].fillna(0)

dfa_alc, dfa_nalc = pd.DataFrame(), pd.DataFrame()

dfa_alc["count"] = dfa.query("alcohol_involved == 1").groupby("year")["case_id"].count() / len(dfa.query("alcohol_involved == 1"))
dfa_alc["alcohol"] = True

dfa_nalc["count"] = dfa.query("alcohol_involved == 0").groupby("year")["case_id"].count() / len(dfa.query("alcohol_involved != 1"))
dfa_nalc["alcohol"] = False

dfap = pd.concat([dfa_nalc, dfa_alc]).reset_index().query("year != 2021")

alt.Chart(dfap).mark_bar(opacity = 0.75, width = 41).encode(
    x = alt.X("year").scale(domain = [2000.5, 2020.5]),
    y = alt.Y("count").stack(None),
    color = "alcohol"
).properties(width = 800, height = 300).display()

alt.Chart(...)

In [95]:
dfs = pd.DataFrame(index = ["overall", "LA only", "SD only", "alcohol only", "no alcohol only", "2019 only", "2020 only"], columns = ["number of collisions", "number of injuries", "average injuries per collision"])

queries = ["case_id == case_id", "county_location == 'los angeles'", "county_location == 'san diego'", "alcohol_involved == 1", "alcohol_involved != 1", "year == 2019", "year == 2020"]

dfs.loc["overall", "number of collisions"] = len(dfy.query("case_id == case_id")["case_id"])

dfs.index

for i, row in enumerate(dfs.index):
    dfs.loc[row, "number of collisions"] = len(dfy.query(queries[i])["case_id"])
    dfs.loc[row, "number of injuries"] = dfy.query(queries[i])["injured_victims"].sum().astype(int)
    dfs.loc[row, "average injuries per collision"] = dfs.loc[row, "number of injuries"] / dfs.loc[row, "number of collisions"]

dfs

,number of collisions,number of injuries,average injuries per collision
overall,3387521,2065489,0.609735
LA only,2851925,1654185,0.580024
SD only,535596,411304,0.767937
alcohol only,313129,195775,0.625222
no alcohol only,3074392,1869714,0.608157
2019 only,175364,107761,0.614499
2020 only,131344,79194,0.602951


In [112]:
dfi_alc, dfi_nalc = pd.DataFrame(), pd.DataFrame()

dfi_alc["count"] = dfc.query("alcohol_involved == 1")["injured_victims"].value_counts()
dfi_alc["pcount"] = dfi_alc["count"] / len(dfc.query("alcohol_involved == 1")["case_id"])
dfi_alc["injured_victims"] = dfc.query("alcohol_involved == 1")["injured_victims"].value_counts().index
dfi_alc["alcohol"] = True

dfi_nalc["count"] = dfc.query("alcohol_involved != 1")["injured_victims"].value_counts()
dfi_nalc["pcount"] = dfi_nalc["count"] / len(dfc.query("alcohol_involved != 1")["case_id"])
dfi_nalc["injured_victims"] = dfc.query("alcohol_involved != 1")["injured_victims"].value_counts().index
dfi_nalc["alcohol"] = False

dfia = pd.concat([dfi_nalc, dfi_alc])

injuries_alc = alt.Chart(dfia).mark_bar(opacity = 0.75, width = 11).encode(
    x = alt.X("injured_victims:Q"),
    y = alt.Y("count:Q").stack(None).scale(type = "log"),
    color = "alcohol"
).properties(width = 800, height = 300)

(injuries_alc).display()

pinjuries_alc = alt.Chart(dfia).mark_bar(opacity = 0.75, width = 11).encode(
    x = alt.X("injured_victims:Q"),
    y = alt.Y("pcount:Q").stack(None).scale(type = "log"),
    color = "alcohol"
).properties(width = 800, height = 300)

(pinjuries_alc).display()

alt.Chart(...)

alt.Chart(...)

In [128]:
pd.DataFrame(df.columns.values.reshape(15, 5))

,0,1,2,3,4
0,case_id,jurisdiction,officer_id,reporting_district,chp_shift
1,population,county_city_location,county_location,special_condition,beat_type
2,chp_beat_type,city_division_lapd,chp_beat_class,beat_number,primary_road
3,secondary_road,distance,direction,intersection,weather_1
4,weather_2,state_highway_indicator,caltrans_county,caltrans_district,state_route
5,route_suffix,postmile_prefix,postmile,location_type,ramp_intersection
6,side_of_highway,tow_away,collision_severity,killed_victims,injured_victims
7,party_count,primary_collision_factor,pcf_violation_code,pcf_violation_category,pcf_violation
8,pcf_violation_subsection,hit_and_run,type_of_collision,motor_vehicle_involved_with,pedestrian_action
9,road_surface,road_condition_1,road_condition_2,lighting,control_device


'case_id', 'jurisdiction', 'officer_id', 'reporting_district',
'chp_shift', 'population', 'county_city_location', 'county_location',
'special_condition', 'beat_type', 'chp_beat_type', 'city_division_lapd',
'chp_beat_class', 'beat_number', 'primary_road', 'secondary_road',
'distance', 'direction', 'intersection', 'weather_1', 'weather_2',
'state_highway_indicator', 'caltrans_county', 'caltrans_district',
'state_route', 'route_suffix', 'postmile_prefix', 'postmile',
'location_type', 'ramp_intersection', 'side_of_highway', 'tow_away',
'collision_severity', 'killed_victims', 'injured_victims',
'party_count', 'primary_collision_factor', 'pcf_violation_code',
'pcf_violation_category', 'pcf_violation', 'pcf_violation_subsection',
'hit_and_run', 'type_of_collision', 'motor_vehicle_involved_with',
'pedestrian_action', 'road_surface', 'road_condition_1',
'road_condition_2', 'lighting', 'control_device', 'chp_road_type',
'pedestrian_collision', 'bicycle_collision', 'motorcycle_collision',
'truck_collision', 'not_private_property', 'alcohol_involved',
'statewide_vehicle_type_at_fault', 'chp_vehicle_type_at_fault',
'severe_injury_count', 'other_visible_injury_count',
'complaint_of_pain_injury_count', 'pedestrian_killed_count',
'pedestrian_injured_count', 'bicyclist_killed_count',
'bicyclist_injured_count', 'motorcyclist_killed_count',
'motorcyclist_injured_count', 'primary_ramp', 'secondary_ramp',
'latitude', 'longitude', 'collision_date', 'collision_time',
'process_date'